# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [ ]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [5]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010, driving for the Red Bull Racing team. It was his first of four championships during his career.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [6]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

The 2011 F1 World Championship was won by Sebastian Vettel, who drove for the Red Bull Racing Team.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [7]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 championship was won by Fernando Alonso driving for Renault.


In [8]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [9]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [10]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

### Results

The Apollo 11 response was well-structured and factual but lacked deeper technical details.

The Mars colonization response covered key challenges like radiation, resources, and mental health but didn’t include recent advancements.

The moon landing controversy response was clear and logical, providing strong counterarguments, but didn’t analyze why the conspiracy theory persists.

### Where GPT Struggled

It did not include recent technology updates unless explicitly asked.

No hallucinations were found, but some answers could be more detailed.

Responses were factual but lacked critical analysis or predictions about the future.

### What I Learned

GPT-3.5 follows structured prompts well, especially with step-by-step formats.

Asking for real-world examples or current research improves responses.

Adding context or specific constraints helps refine the output for better insights.

In [14]:
# Define context for testing new prompts in space exploration
context_space = [{'role': 'system', 'content': 'You are a space exploration expert.'}]

# Mission Breakdown Prompt
context_mission = context_space.copy()
context_mission.append({'role': 'user', 'content': 
    "Explain in a step-by-step manner how the Apollo 11 mission successfully landed on the Moon."})
print(" Mission Breakdown Response:\n", return_OAIResponse("How did Apollo 11 land on the Moon?", context_mission), "\n")


 Mission Breakdown Response:
 1. **Launch**: The Apollo 11 mission launched from Kennedy Space Center in Florida on July 16, 1969, aboard a Saturn V rocket.

2. **Earth Orbit**: After reaching Earth orbit, the spacecraft traveled towards the Moon for about three days.

3. **Trans-Lunar Injection**: Once the spacecraft was on the correct trajectory, the third-stage rocket fired again to propel the spacecraft towards the Moon. 

4. **Approaching the Moon**: The spacecraft entered the Moon's orbit and performed maneuvers to prepare for landing.

5. **Lunar Module Separation**: The lunar module, named Eagle, separated from the command module, Columbia, carrying astronauts Neil Armstrong and Edwin "Buzz" Aldrin.

6. **Descent**: The lunar module began its descent to the Moon's surface, with astronaut Michael Collins remaining in the command module in orbit.

7. **Landing**: On July 20, 1969, the lunar module successfully landed on the Moon's surface in the Sea of Tranquility.

8. **Neil Arm

In [15]:

# Future Space Exploration Prompt
context_future = context_space.copy()
context_future.append({'role': 'user', 'content': 
    "What are the biggest challenges humans will face when colonizing Mars?"})
print("Future Space Exploration Response:\n", return_OAIResponse("What are the biggest challenges of colonizing Mars?", context_future), "\n")

Future Space Exploration Response:
 Colonizing Mars presents a multitude of challenges, some of which include: 

1. **Radiation Exposure**: Mars has a very thin atmosphere that offers much less protection from harmful cosmic and solar radiation compared to Earth. Long-term exposure to this radiation can be damaging to human health.

2. **Harsh Environment**: Mars' extreme temperatures, low atmospheric pressure, and the presence of toxic soil pose significant challenges for human habitation. Building suitable habitats and infrastructure to support life in such conditions is crucial.

3. **Water and Food**: Mars is a barren planet with no readily available sources of water or food. Finding ways to extract and utilize water from Martian resources, as well as establishing sustainable food production systems, will be critical to long-term survival.

4. **Supply Chain Reliability**: The distance between Mars and Earth means that supply missions will be infrequent and potentially unreliable. 

In [16]:

# Space Controversies Prompt
context_controversy_space = context_space.copy()
context_controversy_space.append({'role': 'user', 'content': 
    "What is the most debated controversy in space exploration history?"})
print("Space Controversy Response:\n", return_OAIResponse("What is the biggest space exploration controversy?", context_controversy_space), "\n")


Space Controversy Response:
 One of the most debated controversies in space exploration history is whether or not the United States' Apollo moon landings were real or if they were staged. Some conspiracy theorists believe that the moon landings were faked by NASA in order to win the Space Race against the Soviet Union during the Cold War. Despite overwhelming evidence supporting the authenticity of the moon landings, including samples of moon rocks brought back to Earth, photographs, and testimonies from astronauts and scientists involved in the missions, the conspiracy theories persist and continue to spark debates among skeptics and supporters alike. 

